In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = ""

In [2]:
import sys

sys.path.append("../../app/")
sys.path.append("../../")

# Metrics

In [3]:
import numpy as np
from sklearn.metrics import mean_absolute_error


def accuracy_at_threshold(gold_scores, pred_scores, threshold):
    gold_scores = np.array(gold_scores)
    pred_scores = np.array(pred_scores)
    absolute_errors = np.abs(gold_scores - pred_scores)
    within_threshold_count = np.sum(absolute_errors <= threshold)
    return within_threshold_count / len(gold_scores)


# Evaluate

In [4]:
import pandas as pd

train_df = pd.read_csv("../../data/train.csv")
val_df = pd.read_csv("../../data/test.csv")

## Kaggle dataset

In [5]:
import uuid

from langchain.chat_models import init_chat_model
from langgraph.graph import END, START, StateGraph

import app.workflow.node as node
from app.agents.evaluation import EvaluationAgent
from app.agents.jd_extractor import JDExtractor
from app.agents.resume_extractor import ResumeExtractor
from app.workflow.state import State

model = init_chat_model(
    model="gemini-2.5-flash", model_provider="google_genai", temperature=0
)
RESUME_EXTRACTOR = ResumeExtractor(llm=model)
EVALUATION_AGENT = EvaluationAgent(llm=model)
JD_EXTRACTOR = JDExtractor(llm=model)

CONFIGS = [
    {"obj": node.ValidateInputNode, "configs": {"name": "VALIDATION_INPUT"}},
    {
        "obj": node.ParseContentNode,
        "configs": {
            "name": "CONTENT_EXTRACTION",
            "jd_extractor": JD_EXTRACTOR,
            "resume_extractor": RESUME_EXTRACTOR,
        },
    },
    {
        "obj": node.EvaluatePairMatchingNode,
        "configs": {"name": "EVALUATION", "evaluation_agent": EVALUATION_AGENT},
    },
]

NODES: list[node.BaseNode] = [value["obj"](**value["configs"]) for value in CONFIGS]


def build_graph():
    graph = StateGraph(state_schema=State)
    for user_node in NODES:
        graph.add_node(user_node.name, user_node)

    graph.add_edge(START, "VALIDATION_INPUT")
    graph.add_edge("VALIDATION_INPUT", "CONTENT_EXTRACTION")
    graph.add_edge("CONTENT_EXTRACTION", "EVALUATION")
    graph.add_edge("EVALUATION", END)
    return graph


/Users/nhannguyen/Documents/HCMUS/LLM/Code/llm-resume-evaluation/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1760458976.410134 25809150 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [6]:
import time

from tqdm import tqdm

gold_scores = []
pred_scores = []
graph = build_graph()
app = graph.compile(checkpointer=None)

print(f"\n--- Starting Evaluation on {len(val_df)} examples ---")
batch_size = 4
for i in tqdm(
    range(0, len(val_df), batch_size),
    desc="Evaluating LangGraph in Batches",
    total=len(val_df) // batch_size,
):
    batch_df = val_df.iloc[i : i + batch_size]
    batch_gold_scores = batch_df["match_score"].tolist()

    # Run the LangGraph app
    state = State(
        inputs=[
            {
                "job_description": example["job_description"],
                "resume": example["resume"],
            }
            for _, example in batch_df.iterrows()
        ]
    )
    try:
        result = app.invoke(state)
        batch_pred_scores = [
            result["results"][j].get("score") for j in range(len(result["results"]))
        ]
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        batch_pred_scores = [0.0] * len(batch_df)  # Assign a penalty score

    gold_scores.extend(batch_gold_scores)
    pred_scores.extend(batch_pred_scores)

    # To avoid rate limiting
    time.sleep(1)

print("--- Evaluation Complete ---")



--- Starting Evaluation on 3000 examples ---


Evaluating LangGraph in Batches:   0%|          | 0/750 [00:00<?, ?it/s]

[VALIDATION_INPUT] Validating input...


Evaluating LangGraph in Batches:   0%|          | 1/750 [00:50<10:32:59, 50.71s/it]

[VALIDATION_INPUT] Validating input...


...

[VALIDATION_INPUT] Validating input...


Evaluating LangGraph in Batches: 100%|█████████▉| 749/750 [8:55:23<00:34, 34.65s/it]

[VALIDATION_INPUT] Validating input...


Evaluating LangGraph in Batches: 100%|██████████| 750/750 [8:56:00<00:00, 42.88s/it]

--- Evaluation Complete ---


In [7]:
# missing_index = 754
# pred_scores.insert(missing_index, 5)

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")


--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 1.7247
---
🎯 Accuracy@1 (error <= 1.0): 53.27%
🎯 Accuracy@2 (error <= 2.0): 76.57%
🎯 Accuracy@3 (error <= 3.0): 89.80%
--------------------------------------


In [8]:
import json

with open("preds/preds_ee_gemini_kaggle.json", "w") as f:
    json.dump(pred_scores, f)

## Collected dataset

In [9]:
import json
import time

from tqdm import tqdm

collected_folder = "/Users/nhannguyen/ngtuthanhan@gmail.com - Google Drive/My Drive/HCMUS/LLM/Data_Collector_30_8"
matching_json = os.path.join(collected_folder, "Matching_Result.json")

with open(matching_json, "r") as f:
    matching_data = json.load(f)

pred_scores = []
gold_scores = []

batch_size = 1
for i in tqdm(
    range(0, len(matching_data), batch_size),
    desc="Evaluating LangGraph in Batches",
    total=len(matching_data) // batch_size,
):
    batch_examples = matching_data[i : i + batch_size]
    batch_gold_scores = [example["Score"] for example in batch_examples]
    batch_jd = []
    batch_resume = []
    for example in batch_examples:
        jd_path = example["JD"]
        resume_path = example["CV"]
        gold_score = example["Score"]
        with open(os.path.join(collected_folder, "JD", jd_path) + ".txt", "r") as f:
            jd = f.read()
        with open(os.path.join(collected_folder, "CV", resume_path) + ".txt", "r") as f:
            resume = f.read()
        batch_jd.append(jd)
        batch_resume.append(resume)

    state = State(
        inputs=[
            {
                "job_description": jd,
                "resume": resume,
            }
            for jd, resume in zip(batch_jd, batch_resume)
        ]
    )
    try:
        result = app.invoke(state)
        pred_score = result["results"][0].get("score")
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        pred_score = 0.0  # Assign a penalty score

    gold_scores.append(gold_score)
    pred_scores.append(pred_score)

    # To avoid rate limiting
    time.sleep(1)


print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")

Evaluating LangGraph in Batches:   0%|          | 0/87 [00:00<?, ?it/s]

[VALIDATION_INPUT] Validating input...


Evaluating LangGraph in Batches:   1%|          | 1/87 [00:42<1:01:02, 42.58s/it]

[VALIDATION_INPUT] Validating input...


Evaluating LangGraph in Batches:   2%|▏         | 2/87 [01:34<1:08:01, 48.02s/it]

...

[VALIDATION_INPUT] Validating input...


Evaluating LangGraph in Batches:  99%|█████████▉| 86/87 [1:21:39<00:57, 57.35s/it]

[VALIDATION_INPUT] Validating input...


Evaluating LangGraph in Batches: 100%|██████████| 87/87 [1:22:33<00:00, 56.94s/it]

--- Evaluation Complete ---

--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 2.5057
---
🎯 Accuracy@1 (error <= 1.0): 42.53%
🎯 Accuracy@2 (error <= 2.0): 56.32%
🎯 Accuracy@3 (error <= 3.0): 66.67%
--------------------------------------


In [10]:
import json

with open("preds/preds_ee_gemini_collected.json", "w") as f:
    json.dump(pred_scores, f)